In [ ]:
import dotenv from "dotenv";
dotenv.config({ path: '.env.development.local' });

import OpenAI from "openai";

// Deno imports
import { Stagehand } from "npm:@browserbasehq/stagehand";
import { z } from "npm:zod";
import { underscoreToCapitalCase, run, handleDropDown, initLocalOpenAI, askYesNoGPT, askGptForOption, initLocalStagehand } from "./utils.js";
import { lettuceCustomerDataStub } from "./data-stubs.js";
import { formationsData } from "./customerPayrollData.js";
import { generateStateAddress } from "./index.js";

const cdpPort = 9222;

const page = await initLocalStagehand();

const GPT = new OpenAI({
    apiKey: process.env.OPENAI_API_KEY,
});

let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';

In [ ]:

//Santa Clara???
let data = lettuceCustomerDataStub
data.businessAddress = await generateStateAddress('Florida')
data.mailingAddress = await generateStateAddress('Florida')

console.log(data.businessAddress)
console.log(data.mailingAddress)


In [ ]:

console.log(data)

In [ ]:
const step = async () => {
  await page.goto('https://taxapps.floridarevenue.com/taxregistration');
  await page.waitForTimeout(3_000);

  await page.act("Click on the 'Create User Profile' button");

}

run(step);

In [ ]:

const step = async () => {
    let user = "skunkworksautoFL2";
    let email = "skunkworksauto0+FL2@gmail.com";
    let pass = "F0rm@tions";

    let username_Boxes = await page.observe("Find the 'Required Username' texbox and the 'Confirm Username' textbox");
    let email_Boxes = await page.observe("Find the 'Required Email Address' texbox and the 'Confirm Email Addres' textbox");
    let password_Boxes = await page.observe("Find the 'Required Password' texbox and the 'Confirm Password' textbox");

    username_Boxes[0].arguments = [user];
    username_Boxes[1].arguments = [user];

    email_Boxes[0].arguments = [email];
    email_Boxes[1].arguments = [email];

    password_Boxes[0].arguments = [pass];
    password_Boxes[1].arguments = [pass];
    
    await page.act(username_Boxes[0]);
    await page.act(username_Boxes[1]);

    await page.act(email_Boxes[0]);
    await page.act(email_Boxes[1]);

    await page.act(password_Boxes[0]);
    await page.act(password_Boxes[1]);

    await page.act("Click on the 'Create New User Profile' Button");
}

run(step);

In [ ]:

const verify = async () => {
    let email = "skunkworksauto0@gmail.com"
    let password = "Lettuce99!"

    await page.goto('https://accounts.google.com');
    await page.waitForTimeout(3_000);
    
    await page.act(`Fill out the "Email or phone" textbox with ${email}`);
    await page.act("Click on the 'Next' Button");
    await page.waitForTimeout(3_000);
    
    await page.act(`Fill out the "Enter your password" textbox with ${password}`);
    await page.act("Click on the 'Next' Button");
    await page.waitForTimeout(3_000);

    await page.goto('https://mail.google.com/mail/u/0/#inbox');
    await page.waitForTimeout(3_000);
    
    await page.act("Under the PRIMARY section, click on the top-most 'Florida Department' link");
    await page.waitForTimeout(3_000);
    
    await page.act("Click on the extended URL link");
    await page.waitForTimeout(3_000);

    await page.act("Click on the 'Log in' link");
    await page.waitForTimeout(3_000);    
}

run(verify);

In [ ]:

const Login = async () => {
    let user = "skunkworksautoFL2";
    let pass = "F0rm@tions";

    await page.act(`Fill out the "Username" textbox with ${user}`);
    await page.act(`Fill out the "Enter your password" textbox with ${pass}`);

    await page.act("Click on the 'Login' Button");
}

run(Login);

In [ ]:

const Step = async () => {

    await page.act("Click on the 'Start New Application' Button");
    
}

run(Step);

In [ ]:

const Step = async () => {

    await handleDropDown("Find the 'Reason for Applying' drop down", 'New' );
    await page.waitForTimeout(2_000);

    await page.act(`Fill out the "Date of first Florida taxable activity" textbox with ${data.dateFiled}`);
    await page.act("Click on the 'Save & Continue' button");
}

run(Step);

In [ ]:

const Step = async () => {

    await page.act(`Fill out the "Legal Name of Business" textbox with ${data.organizationLegalName}`);

    await page.act(`Fill out the "Telephone Number" textbox with ${data.businessPhoneNumber}`);

    await page.act("Click on the 'No' checkbox");

    await page.act("Click on the 'Save & Continue' button");
}

run(Step);

In [ ]:
const Step = async () => {

    let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
    
    let q = await page.extract("Find the first statement corresponding to Yes/No buttons on the page");
    let q2 = await page.extract("Find the second statement corresponding to Yes/No buttons on the page");
   
    let options = await page.observe(`Find the Yes/No radio buttons for the FIRST question: '${q}'`);
    let options2 = await page.observe(`Find the Yes/No radio buttons for the SECOND question: '${q2}'`);

    console.log(options)
    console.log(options2)

    let answer = await askYesNoGPT(description, q, GPT);
    let answer2 = await askYesNoGPT(description, q2, GPT);

    await page.act(options[answer]);

    await page.act(options2[answer2]);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {

    await handleDropDown("Find the 'Florida County' drop down", data.businessAddress.county);
   
    let businessAddress_modal = await page.observe("In the 'Physical Location of Business or Rental Property' section, find the Street Address, City and Postal Code textboxes");
    let mailingAddress_modal = await page.observe("In the 'Business Mailing Address' section, find the Street Address, City and Postal Code textboxes");

    let bStreet = businessAddress_modal[0];
    let bCity = businessAddress_modal[1];
    let bPostal = businessAddress_modal[2];

    let mStreet = mailingAddress_modal[0];
    let mCity = mailingAddress_modal[1];
    let mPostal = mailingAddress_modal[2];

    bStreet.arguments = [data.businessAddress.street1];
    bCity.arguments = [data.businessAddress.city];
    bPostal.arguments = [data.businessAddress.postal + "1234"];

    mStreet.arguments = [data.businessAddress.street1]
    mCity.arguments = [data.businessAddress.city]
    mPostal.arguments = [data.businessAddress.postal + "1234"];

    await page.act(bStreet);
    await page.act(bCity);
    await page.act(bPostal);

    await page.act(mStreet);
    await page.act(mCity);
    await page.act(mPostal);

    await page.act("Click on the 'Save & Continue' button");
    await page.waitForTimeout(3_000);

    let confirm_modal = await page.observe("There are two 'I confirm the address entered is correct' checkboxes, find them.");

    await page.act(confirm_modal[0]);
    await page.act(confirm_modal[1]);

        //Differentiate better with just act
    /*
    await page.act("In the 'Physical Location of Business or Rental Property' section, click on the 'I confirm the address entered is correct' checkbox");
    await page.act("In the 'Business Mailing Address' section, click on the 'I confirm the address entered is correct' checkbox");
    */

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {

    await page.act("Click on the 'NAICS Code' radio button");

    await page.act("Click on the 'Save & Continue' button");

    //Or should we search for it?
    await page.act("Click on the 'Enter Known NAICS Code' radio button");



}

run(Step);